In [ ]:
%load_ext autoreload

import os
import shutil
import sys
import numpy as np
import pandas as pd
import xarray as xr

sys.path.append("/glade/u/home/bbuchovecky/ctsm6_ppe/gen_ensembles/gen_paramfiles")
from ppe_tools import Ensemble

%aimport ppe_tools
%autoreload 2

# Generate parameter perturbations
Remember to address FUN bug in CLM5.0 parameter file -- run `fix_FUN_bug.py` to swap kc_nonmyc and kn_nonmyc. This really only matters when providing the parameter basefile, since the KCN perturbations are defined as the same percent changes.

In [57]:
# Load file with parameter ranges
csv = "coupPPE-hist_params.csv"
params = pd.read_csv(csv, index_col=0)

basedir = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/cesm2.1.5"

# Where to save the parameter files
pdir = f"{basedir}/paramfiles"

# Where to save the namelist mods
ndir = f"{basedir}/nlmods"

# Where to save the source code mods
sdir = f"{basedir}/srcmods/perturbed"

# Where to look for staged source code templates (all instances of '<parameter>' replaced with 'this_<parameter-name>')
staged_sdir = f"{basedir}/srcmods/templates"

# Default parameter file to start with -> addressed FUN bug by swapping 'kn_nonmyc' and 'kc_nonmyc' (see fix_FUN_bug.py)
basefile = f"{pdir}/coupPPE-hist.000.nc"

# Instantiate the Ensemble object
x = Ensemble(basefile, pdir=pdir, ndir=ndir, sdir=sdir, staged_sdir=staged_sdir)

In [59]:
params;

In [60]:
# Create dictionary for the independent parameters
oaats = {}
names = params["name"]
flags = params["flag"]
for name, flag in zip(names, flags):
    if not pd.notnull(flag):
        print(name)
        ix = params["name"] == name
        minval = params["min"][ix].values[0]
        maxval = params["max"][ix].values[0]
        pftmin = params["pft_mins"][ix].values[0]
        pftmax = params["pft_maxs"][ix].values[0]
        thisloc = params["loc"][ix].values[0]

        needs_pft = minval == "pft"
        if needs_pft:
            thismin = np.zeros((79))
            pftmin_np = np.fromstring(pftmin, dtype="float", sep=",")
            thismin[0 : len(pftmin_np)] = pftmin_np
            # thismin[0:17] = np.fromstring(pftmin, dtype='float', sep=',')
        elif isinstance(minval, str) and "percent" in minval:
            thismin = minval
        else:
            thismin = np.array(float(minval))

        needs_pft = maxval == "pft"
        if needs_pft:
            thismax = np.zeros((79))
            pftmax_np = np.fromstring(pftmax, dtype="float", sep=",")
            thismax[0 : len(pftmax_np)] = pftmax_np
            # thismax[0:17] = np.fromstring(pftmax, dtype='float', sep=',')
        elif isinstance(minval, str) and "percent" in maxval:
            thismax = maxval
        else:
            thismax = np.array(float(maxval))

        ## NEW CODE FOR SOURCE MODS
        if thisloc == "S" and "def" in params.keys():
            thisdef = float(params["def"][ix].values[0])
            oaats[name] = {"min": thismin, "max": thismax, "loc": thisloc, "def": thisdef}
        else:
            oaats[name] = {"min": thismin, "max": thismax, "loc": thisloc}

dleaf
d_max
maximum_leaf_wetted_fraction
fff
medlynslope
medlynintercept
Jmaxb0
kmax
psi50
FUN_fracfixers
leafcn
lmrha


In [61]:
# Add to the Ensemble object
prefix = "coupPPE-hist."
nextnum = 1
x.add_oaats(oaats, prefix, nextnum, skipBFB=True)
print(f"number of members (2x number of parameters): {x.nmemb}")

number of members (2x number of parameters): 24


In [62]:
# Add params that move in unison to the Ensemble object
flags = params["flag"]
ix = pd.notnull(params["flag"])
uflags = pd.unique(flags[ix])
sgns = {"min": "-", "max": ""}
for uflag in uflags:
    print(uflag)
    names = params["name"][flags == uflag]
    for minmax in ["min", "max"]:
        mf = {}

        for name in names:
            print(" ", name, minmax)
            ix = params["name"] == name
            thisval = params[minmax][ix].values[0]
            pftval = params["pft_" + minmax + "s"][ix].values[0]
            thisloc = params["loc"][ix].values[0]

            needs_pft = thisval == "pft"
            if needs_pft:
                val = np.fromstring(pftval, dtype="float", sep=",")
            elif "percent" in thisval:
                val = sgns[minmax] + thisval
            else:
                val = np.array(float(thisval))
            
            ## NEW CODE FOR SOURCE MODS            
            if thisloc == "S" and "def" in params.keys():
                thisdef = float(params["def"][ix].values[0])
                mf[name] = {"value": val, "loc": thisloc, "minmax": minmax, "def": thisdef, "flag": uflag}
            else:
                mf[name] = {"value": val, "loc": thisloc, "minmax": minmax, "flag": uflag}

        x.add_mf(mf, prefix)

print(f"number of members (2x number of parameters): {x.nmemb}")

KCN
  kc_nonmyc min
  kn_nonmyc min
  akc_active min
  akn_active min
  ekc_active min
  ekn_active min
  kc_nonmyc max
  kn_nonmyc max
  akc_active max
  akn_active max
  ekc_active max
  ekn_active max
ACCLIM_SF
  vcmaxse_sf min
  jmaxse_sf min
  tpuse_sf min
  vcmaxse_sf max
  jmaxse_sf max
  tpuse_sf max
number of members (2x number of parameters): 28


In [64]:
# x.print_members()

In [ ]:
# os.system(f"rm {pdir}/coupPPE-hist.0[0-9][1-9].nc")
# os.system(f"rm {pdir}/coupPPE-hist.0[1-9]0.nc")

# os.system(f"rm {ndir}/coupPPE-hist.*.txt")

# os.system(f"rm -rf /glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/cesm2.1.5/srcmods/perturbed/coupPPE-hist.*")

0

In [67]:
# Write the parameter files and namelist mods 
csvfile= "coupPPE-hist_OAAT.csv"
os.remove(csvfile)
x.write(oaatfile=csvfile)
shutil.copy2(csvfile, f"{basedir}/{csvfile}")

'/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/cesm2.1.5/coupPPE-hist_OAAT.csv'

In [68]:
pd.read_csv('coupPPE-hist_OAAT.csv', names=["name", "param", "minmax"])

,name,param,minmax
0,coupPPE-hist.001,dleaf,min
1,coupPPE-hist.002,dleaf,max
2,coupPPE-hist.003,d_max,min
3,coupPPE-hist.004,d_max,max
4,coupPPE-hist.005,maximum_leaf_wetted_fraction,min
5,coupPPE-hist.006,maximum_leaf_wetted_fraction,max
6,coupPPE-hist.007,fff,min
7,coupPPE-hist.008,fff,max
8,coupPPE-hist.009,medlynslope,min
9,coupPPE-hist.010,medlynslope,max


# Create csv file

Remember to address FUN bug in CLM5.0 parameter file -- run `fix_FUN_bug.py` to swap kc_nonmyc and kn_nonmyc. This really only matters when providing the parameter basefile for the perturbation code above, since the KCN perturbations are defined as percent changes.

There is a discrepancy with Jmaxb0 in the `clm5_paramranges.csv` file where Jmaxb0 is written as jmaxb0 in the parameter file but has a capital 'J' in the source code. I changed 'jmaxb0' to 'Jmaxb0' in `clm5_paramranges.csv` so the following code will work.

In [25]:
clm5x_paramfile_path = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/cesm2.2.0_PPE.n11/paramfiles/COUP0000.nc"
clm5x_paramfile = xr.open_dataset(clm5x_paramfile_path, decode_timedelta=False)

clm50_paramfile_path = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/hist/data/cesm2.1.5/paramfiles/clm5_params.c171117_FUNfix.nc"
clm50_paramfile = xr.open_dataset(clm50_paramfile_path, decode_timedelta=False)

In [26]:
clm5_csv = pd.read_csv("clm5_paramranges.csv")

clm6_csv = pd.read_csv("ctsm6lhc_paramranges_11262024.csv")
clm6_params_set = set(clm6_csv.param)

In [27]:
params_list = [
    "fff",
    "Jmaxb0",
    "medlynslope",
    "medlynintercept",
    "maximum_leaf_wetted_fraction",
    "kmax",
    "leafcn",
    "d_max",
    "psi50",
    "dleaf",  # instead of cv
    "lmrha",
    "FUN_fracfixers",
    "jmaxse_sf",
    "tpuse_sf",
    "vcmaxse_sf",
    "akc_active",
    "akn_active",
    "ekc_active",
    "ekn_active",
    "kc_nonmyc",
    "kn_nonmyc",
    # "sand_pf",
]

KCN_group = [
    "akc_active",
    "akn_active",
    "ekc_active",
    "ekn_active",
    "kc_nonmyc",
    "kn_nonmyc",
]

ACCLIM_SF_group = [
    "jmaxse_sf",
    "tpuse_sf",
    "vcmaxse_sf",
]

use_constrained_clm6_values = [
    "medlynintercept",
    "FUN_fracfixers",
]

hardcoded_params_in_clm50 = [
    "d_max",
    "fff",
    "jmaxse_sf",
    "lmrha",
    "tpuse_sf",
    "vcmaxse_sf",
    "Jmaxb0",
]
print("hard-coded parameters in CLM5.0.37: ", hardcoded_params_in_clm50)

hard-coded parameters in CLM5.0.37:  ['d_max', 'fff', 'jmaxse_sf', 'lmrha', 'tpuse_sf', 'vcmaxse_sf', 'Jmaxb0']


In [28]:
params_csv = clm5_csv[clm5_csv['name'].isin(params_list)]
params_csv.loc[params_csv['name'].isin(hardcoded_params_in_clm50), 'location'] = 'S'
params_csv.loc[params_csv['name'].isin(ACCLIM_SF_group), 'flag'] = 'ACCLIM_SF'
params_csv = params_csv[["name", "location", "flag", "min", "max", "Description", "category", "subcat", "CLM5 Default Value(s)", "pft_mins", "pft_maxs"]]
params_csv = params_csv.rename(columns={"CLM5 Default Value(s)": "def", "location": "loc", "Description": "description"}).reset_index(drop=True)
params_csv["notes"] = ""

# Use constrained values from CLM6
for p in use_constrained_clm6_values:
    p_clm6 = clm6_csv.loc[clm6_csv.param==p]
    params_csv.loc[params_csv['name']==p, 'min'] = p_clm6["min"].values
    params_csv.loc[params_csv['name']==p, 'max'] = p_clm6["max"].values
    params_csv.loc[params_csv['name']==p, 'notes'] = "use constrained range from CLM6"

params_csv.to_csv("coupPPE-hist_params.csv")

In [38]:
params_csv;

In [31]:
for p in params_list:
    p_clm6 = clm6_csv.loc[clm6_csv.param==p]
    p_clm5 = clm5_csv.loc[clm5_csv.name==p]
    p_clm5 = p_clm5[["name", "location", "flag", "min", "max", "Description", "category", "subcat", "CLM5 Default Value(s)", "pft_mins", "pft_maxs"]]
    p_clm5 = p_clm5.rename(columns={"CLM5 Default Value(s)": "def", "location": "loc", "Description": "description"}).reset_index(drop=True)

In [32]:
p_clm6

,param,loc,include,flag,min,max,pft_mins,pft_maxs
29,kn_nonmyc,P,0,KCN,pft,pft,"0, 0.072, 0.072, 0.072, 0.072, 0.072, 0.0072, ...","0, 7.2, 7.2, 7.2, 7.2, 7.2, 0.72, 7.2, 7.2, 7...."


In [33]:
p_clm5

,name,loc,flag,min,max,description,category,subcat,def,pft_mins,pft_maxs
0,kn_nonmyc,P,KCN,90percent,900percent,Constant relating soil layer Nitrogen content ...,bgc,fun,NaN,NaN,NaN
